In [ ]:
# Install required packages
!pip install kagglehub -q

In [ ]:
# Import libraries
import pandas as pd
import os
import numpy as np
import tensorflow as tf
import cv2
import kagglehub

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# Download dataset (this might take a few minutes)
print("Downloading dataset...")
dataset_path = kagglehub.dataset_download('bjoernjostein/food-classification')
print(f"Dataset downloaded to: {dataset_path}")

In [ ]:
# Load training data
training_dataframe = pd.read_csv(f"{dataset_path}/train_img.csv")
print(f"Training samples: {training_dataframe.shape[0]}")
print(f"Classes: {training_dataframe['ClassName'].nunique()}")

# Prepare data
class_names = pd.get_dummies(training_dataframe["ClassName"]).columns
y_dev = np.asarray(pd.get_dummies(training_dataframe["ClassName"]))
X_dev = np.asarray(training_dataframe["ImageId"])
train_path = f"{dataset_path}/train_images/train_images/"

In [ ]:
# Data generators
def batch_generator(batch_size, gen_x):
    batch_features = np.zeros((batch_size, 256, 256, 3))
    batch_labels = np.zeros((batch_size, 61))
    while True:
        for i in range(batch_size):
            batch_features[i], batch_labels[i] = next(gen_x)
        yield batch_features, batch_labels

def generate_data(filelist, img_path, target):
    while True:
        for i, j in enumerate(filelist):
            X_train = cv2.imread(img_path + j, cv2.IMREAD_COLOR)
            X_train = cv2.resize(X_train, (256, 256), interpolation=cv2.INTER_LINEAR)
            y_train = target[i]
            yield X_train, y_train

In [ ]:
# Create model
def create_model():
    base_model = tf.keras.applications.EfficientNetB1(
        weights="imagenet", include_top=False, input_shape=(256, 256, 3)
    )
    num_classes = 61

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=predictions)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="categorical_crossentropy",
        metrics=['accuracy']
    )
    return model

model = create_model()
print("Model created successfully!")

In [ ]:
# Train model (15 epochs will take ~25-30 minutes with GPU for better accuracy)
batch_size = 64
num_epoch = 15  # Increased from 5 to 15 for better accuracy

print("Starting training...")
print("⏰ This will take 25-30 minutes with GPU...")
history = model.fit(
    x=batch_generator(batch_size, generate_data(X_dev, train_path, y_dev)),
    epochs=num_epoch,
    steps_per_epoch=int(y_dev.shape[0] / batch_size),
    verbose=1
)
print("✅ Training complete!")

In [ ]:
# Save model
model.save('food_classification_model.keras')
print("\n✅ Model saved as 'food_classification_model.keras'")
print("\nNext steps:")
print("1. Download this file from Colab (click the folder icon on left → right-click file → Download)")
print("2. Move it to: C:\\Users\\sushe\\OneDrive\\Desktop\\sushi\\backend\\")
print("3. Restart your backend server")
print("4. Test predictions!")